<a href="https://colab.research.google.com/github/Knight7561/hf-smol-course/blob/learn-module-2/2_preference_alignment/student_examples/orpo_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preference Alignment with Odds Ratio Preference Optimization (ORPO)

This notebook will guide you through the process of fine-tuning a language model using Odds Ratio Preference Optimization (ORPO). We will use the SmolLM2-135M model which has **not** been through SFT training, so it is not compatible with DPO. This means, you cannot use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with ORPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p>
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Try on a subset of mlabonne's `orpo-dpo-mix-40k` dataset</p>
</div>



## Import libraries


In [14]:
!pip install datasets trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.8 MB/s eta 0:00:00


In [5]:
import torch
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

# Authenticate to Hugging Face
# from huggingface_hub import login

# login()

## Format dataset

In [6]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
# dataset = load_dataset(path="trl-lib/ultrafeedback_binarized")
dataset = load_dataset(path="quangduc1112001/python-code-DPO-fine-tune")

README.md:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/870k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/99.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [7]:
# TODO: 🐕 If your dataset is not represented as conversation lists, you can use the `process_dataset` function to convert it.

## Define the model

In [9]:
model_name = "knight7561/SmolLM2_python_coder"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
# disabling line because Chat template is already added to the tokenizer. If you want to overwrite it, please set it to None
# model, tokenizer = setup_chat_format(model, tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2_python_coder-FT-ORPO"
finetune_tags = ["smol-course", "module_1","ORPO"]

## Train model with ORPO

In [16]:
orpo_args = ORPOConfig(
    # Small learning rate to prevent catastrophic forgetting
    learning_rate=8e-6,
    # Linear learning rate decay over training
    lr_scheduler_type="linear",
    # Maximum combined length of prompt + completion
    max_length=1024,
    # Maximum length for input prompts
    max_prompt_length=512,
    # Controls weight of the odds ratio loss (λ in paper)
    beta=0.1,
    # Batch size for training
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # Helps with training stability by accumulating gradients before updating
    gradient_accumulation_steps=4,
    # Memory-efficient optimizer for CUDA, falls back to adamw_torch for CPU/MPS
    optim="paged_adamw_8bit" if device == "cuda" else "adamw_torch",
    # Number of training epochs
    num_train_epochs=1,
    # When to run evaluation
    evaluation_strategy="steps",
    # Evaluate every 20% of training
    eval_steps=0.2,
    # Log metrics every step
    logging_steps=1,
    # Gradual learning rate warmup
    warmup_steps=10,
    # Disable external logging
    report_to='none',
    # Where to save model/checkpoints
    output_dir="./results/",
    # Enable MPS (Metal Performance Shaders) if available
    use_mps_device=device == "mps",
    hub_model_id=finetune_name,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/orpo_trainer.py:275: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [18]:
trainer.train()  # Train the model

# Save the model
trainer.save_model(f"./{finetune_name}")

# # Save to the huggingface hub if login (HF_TOKEN is set)
# if os.getenv("HF_TOKEN"):
#     trainer.push_to_hub(tags=finetune_tags)

Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
45,1.344800,1.402951,12.597500,15.876000,7.938000,-0.131000,-0.216596,0.715000,0.085597,-2.165965,-1.309997,8.464478,8.132378,1.339626,-0.633250,0.946981
90,1.067200,1.265223,12.580800,15.897000,7.949000,-0.118259,-0.222777,0.800000,0.104517,-2.227768,-1.182594,8.258402,8.010843,1.210492,-0.547305,1.210568
135,1.084300,1.208775,12.796100,15.630000,7.815000,-0.110354,-0.231728,0.810000,0.121374,-2.317277,-1.103542,7.912307,7.649135,1.159117,-0.496576,1.435444
180,0.960900,1.190824,12.630200,15.835000,7.918000,-0.107849,-0.236770,0.825000,0.128921,-2.367701,-1.078486,7.815880,7.552517,1.143088,-0.477351,1.532207
225,0.933300,1.185642,12.602700,15.870000,7.935000,-0.107198,-0.238421,0.825000,0.131223,-2.384212,-1.071983,7.795619,7.543156,1.138411,-0.472309,1.561286


In [21]:
trainer.push_to_hub(tags=finetune_tags)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1737415210.9d9d700863ba.1469.0:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/knight7561/SmolLM2_python_coder-FT-ORPO/commit/c1cfa1a563e240d3304dc7b4c100251435761270', commit_message='End of training', commit_description='', oid='c1cfa1a563e240d3304dc7b4c100251435761270', pr_url=None, repo_url=RepoUrl('https://huggingface.co/knight7561/SmolLM2_python_coder-FT-ORPO', endpoint='https://huggingface.co', repo_type='model', repo_id='knight7561/SmolLM2_python_coder-FT-ORPO'), pr_revision=None, pr_num=None)

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `ORPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.